# 本Code執行Model training
* images_0為0_images.npy的路徑
* labels_0為0_labels_hot.npy的路徑
* 以此類推。。。

In [1]:

import tensorflow
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Flatten
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.optimizers import SGD,RMSprop
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# 路徑檔名要改

In [2]:
import numpy as np

images_all=[]
labels_all=[]
#Read cat npy file
images_0=np.load('C:\\Users\\Administrator\\Desktop\\yilin\\temp\\0_images.npy')
images_all=images_0
labels_0=np.load('C:\\Users\\Administrator\\Desktop\\yilin\\temp\\0_labels_hot.npy')
labels_all=labels_0
#Read dog npy file
images_1=np.load('C:\\Users\\Administrator\\Desktop\\yilin\\temp\\1_images.npy')
images_all=np.append(images_all,images_1,axis=0)
labels_1=np.load('C:\\Users\\Administrator\\Desktop\\yilin\\temp\\1_labels_hot.npy')
labels_all=np.append(labels_all,labels_1,axis=0)

images_2=np.load('C:\\Users\\Administrator\\Desktop\\yilin\\temp\\2_images.npy')
images_all=np.append(images_all,images_2,axis=0)
labels_2=np.load('C:\\Users\\Administrator\\Desktop\\yilin\\temp\\2_labels_hot.npy')
labels_all=np.append(labels_all,labels_2,axis=0)

In [3]:
y_one_hot = np.zeros([labels_all.size, np.amax(labels_all)+1])
y_one_hot[np.arange(labels_all.size), labels_all.reshape(1, labels_all.size)]  = 1

# Model 架構

In [4]:

# Generate model
model = Sequential()
# input: 190x190 images with 3 channels -> (224, 224, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(224,224,3),padding='same',name='block1_conv2_1'))
model.add(Conv2D(64, (3, 3), activation='relu',padding='same',name='block1_conv2_2'))
model.add(MaxPooling2D(pool_size=(2, 2),name='block1_MaxPooling'))

model.add(Conv2D(128, (3, 3), activation='relu',padding='same',name='block2_conv2_1'))
model.add(Conv2D(128, (3, 3), activation='relu',padding='same',name='block2_conv2_2'))
model.add(MaxPooling2D(pool_size=(2, 2),name='block2_MaxPooling'))

model.add(Conv2D(256, (3, 3), activation='relu',padding='same',name='block3_conv2_1'))
model.add(Conv2D(256, (3, 3), activation='relu',padding='same',name='block3_conv2_2'))
model.add(Conv2D(256, (3, 3), activation='relu',padding='same',name='block3_conv2_3'))
model.add(Conv2D(256, (3, 3), activation='relu',padding='same',name='block3_conv2_4'))
model.add(Conv2D(256, (3, 3), activation='relu',padding='same',name='block3_conv2_5'))
model.add(MaxPooling2D(pool_size=(2, 2),name='block3_MaxPooling'))


model.add(Conv2D(512, (3, 3), activation='relu',padding='same',name='block4_conv2_1'))
model.add(Conv2D(512, (3, 3), activation='relu',padding='same',name='block4_conv2_2'))
model.add(Conv2D(512, (3, 3), activation='relu',padding='same',name='block4_conv2_3'))
model.add(Conv2D(512, (3, 3), activation='relu',padding='same',name='block4_conv2_4'))
model.add(Conv2D(512, (3, 3), activation='relu',padding='same',name='block4_conv2_5'))
model.add(MaxPooling2D(pool_size=(2, 2),name='block4_MaxPooling'))

model.add(Conv2D(512, (3, 3), activation='relu',padding='same',name='block5_conv2_1'))
model.add(Conv2D(512, (3, 3), activation='relu',padding='same',name='block5_conv2_2'))
model.add(Conv2D(512, (3, 3), activation='relu',padding='same',name='block5_conv2_3'))
model.add(Conv2D(512, (3, 3), activation='relu',padding='same',name='block5_conv2_4'))

model.add(MaxPooling2D(pool_size=(2, 2),name='block5_MaxPooling'))
model.add(Flatten())
model.add(Dense(4096, activation='relu',name='final_output_1'))
model.add(Dense(4096, activation='relu',name='final_output_2'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax',name='class_output'))
model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])

In [5]:

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv2_1 (Conv2D)      (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2_2 (Conv2D)      (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_MaxPooling (MaxPoolin (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv2_1 (Conv2D)      (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2_2 (Conv2D)      (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_MaxPooling (MaxPoolin (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv2_1 (Conv2D)      (None, 56, 56, 256)       2

# 5 fold Cross Validation
* kf=KFold(n_splits=5,shuffle=True)----->執行5folds

In [6]:
from sklearn.model_selection import KFold
kf=KFold(n_splits=5,shuffle=True)
cvscores = []
cvsen=[]
cvspe=[]
for train,test in kf.split(images_all,y_one_hot):
    model.fit(images_all[train], y_one_hot[train], epochs=15, batch_size=57, verbose=1)
    # evaluate the model
    scores = model.evaluate(images_all[test], y_one_hot[test], verbose=1)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    pre_sen=model.predict(images_all[test])
    ##############class report################################
    for i in range(len(pre_sen)):
        max_value=max(pre_sen[i])
        for j in range(len(pre_sen[i])):
            if max_value==pre_sen[i][j]:
                pre_sen[i][j]=1
            else:
                pre_sen[i][j]=0
    from sklearn.metrics import classification_report
    label2index={'NO DR':0,'Moderate DR':1,'Severe DR':2}
    res=(classification_report(y_one_hot[test],pre_sen,target_names=label2index,digits=4))
    print(res)
    ################Kappa#################################################
    from sklearn.metrics import cohen_kappa_score,confusion_matrix
    kappa = cohen_kappa_score(y_one_hot[test].argmax(axis=1),pre_sen.argmax(axis=1),weights='quadratic')
    print(kappa)
    ###############confusion matrix#####################################
    import seaborn as sns
    import  matplotlib.pyplot as plt
    cm=confusion_matrix(y_one_hot[test].argmax(axis=1),pre_sen.argmax(axis=1))
    LABELS=['NO DR','Moderate DR','Severe DR']
    cmn = cm.astype('float')/cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(30, 30))
    sns.heatmap(cmn, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt=".2f");
    plt.title("Confusion matrix")
    plt.ylabel('True class')
    plt.xlabel('Predicted class')
    plt.show()
    ######################################################################
    m=tensorflow.python.keras.metrics.SensitivityAtSpecificity(0.5)
    m.update_state(y_one_hot[test],pre_sen)
    mm=m.result().numpy()
    mmm = tensorflow.python.keras.metrics.SpecificityAtSensitivity(0.5)
    mmm.update_state(y_one_hot[test],pre_sen)
    mmmm=mmm.result().numpy()
    cvsen.append(mm)
    cvspe.append(mmmm)
    cvscores.append(scores[1] * 100)
print("Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("Sensitivity %.2f%% (+/- %.2f%%)" % (np.mean(cvsen)*100, np.std(cvsen)*100))
print("Specificity %.2f%% (+/- %.2f%%)" % (np.mean(cvspe)*100, np.std(cvspe)*100))
    

Epoch 1/15
59/59 [==============================] - 23s 382ms/step - loss: 1.0831 - accuracy: 0.4385
Epoch 2/15
59/59 [==============================] - 22s 373ms/step - loss: 1.0508 - accuracy: 0.4493
Epoch 3/15
59/59 [==============================] - 22s 375ms/step - loss: 1.0989 - accuracy: 0.3840
Epoch 4/15
59/59 [==============================] - 22s 376ms/step - loss: 1.0916 - accuracy: 0.4830
Epoch 5/15
59/59 [==============================] - 22s 377ms/step - loss: 1.0702 - accuracy: 0.4833
Epoch 6/15
59/59 [==============================] - 22s 378ms/step - loss: 1.0506 - accuracy: 0.4965
Epoch 7/15
59/59 [==============================] - 22s 378ms/step - loss: 1.0741 - accuracy: 0.5339
Epoch 8/15
59/59 [==============================] - 22s 378ms/step - loss: 0.9808 - accuracy: 0.4977
Epoch 9/15
59/59 [==============================] - 22s 379ms/step - loss: 0.7509 - accuracy: 0.6389
Epoch 10/15
59/59 [==============================] - 22s 380ms/step - loss: 0.6587 - accura

Epoch 5/15
59/59 [==============================] - 23s 383ms/step - loss: 0.1046 - accuracy: 0.9513
Epoch 6/15
59/59 [==============================] - 23s 383ms/step - loss: 0.0877 - accuracy: 0.9600
Epoch 7/15
59/59 [==============================] - 23s 383ms/step - loss: 0.1410 - accuracy: 0.9350
Epoch 8/15
59/59 [==============================] - 23s 383ms/step - loss: 0.0893 - accuracy: 0.9603
Epoch 9/15
59/59 [==============================] - 23s 383ms/step - loss: 0.0794 - accuracy: 0.9636
Epoch 10/15
59/59 [==============================] - 23s 383ms/step - loss: 0.0893 - accuracy: 0.9627
Epoch 11/15
59/59 [==============================] - 23s 383ms/step - loss: 0.0551 - accuracy: 0.9753
Epoch 12/15
59/59 [==============================] - 23s 383ms/step - loss: 0.0889 - accuracy: 0.9621
Epoch 13/15
59/59 [==============================] - 23s 382ms/step - loss: 0.1144 - accuracy: 0.9567
Epoch 14/15
59/59 [==============================] - 23s 383ms/step - loss: 0.0956 - ac

In [7]:
print("Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("Sensitivity %.2f%% (+/- %.2f%%)" % (np.mean(cvsen)*100, np.std(cvsen)*100))
print("Specificity %.2f%% (+/- %.2f%%)" % (np.mean(cvspe)*100, np.std(cvspe)*100))

Accuracy: 81.08% (+/- 9.01%)
Sensitivity 81.08% (+/- 9.01%)
Specificity 90.54% (+/- 4.51%)
